# CNN Trainig Templete
## Based on Tensorlfow - CNN MNIST example
https://github.com/tensorflow/tensorflow/blob/r1.6/tensorflow/examples/tutorials/layers/cnn_mnist.py

Tapping modified

## 필요한 library 호출

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)


## CNN model 설계
현재는 MNIST에 사용된 것으로, 추후 그물 찢김에 적용하기 위해 수정 필요

### Input Layer
Input Image Pixel: 28x28  
Color channel: 1 (grayscale)  

### 1st Conv Layer
Num of filter: 32  
Kernel size: 5x5  
Padding: Same  
Activation: ReLU  
Padding same은 입/출력이 같은 크기를 갖도록 padding하는 것  

### 1st Pooling Layer
Max pooling  
Size: 2x2  
Stride: 2  
Max pooling 시 size의 폭과 stride가 같으면 pooling window가 서로 겹치지 않음  

### 2nd Conv Layer
Num of filter: 64  
Kernel size: 5x5  
Padding: Same  
Activation: ReLU  

### 2nd Pooling Layer
Max pooling  
Size: 2x2  
Stride: 2  

### Dense Layer
Fully connected  

### Drop out
Rate: 0.4 (40%)  
훈련 중 40%의 connection이 임의로 제거됨  
Overfitting 방지  
뇌의학에서도 학습을 반복할수록 사람 뇌의 뉴런은 일부 connection이 끊어지면서 더 확실하게 학습한다고 함  

### Logits Layer
Output node: 10  
숫자 0~9까지 classification

### Training
Loss function: Softmax Cross Entropy  
Optimizer: Gradient Descent  
Learning rate: 0.001  

In [4]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    # Reshape X to 4-D tensor: [batch_size, width, height, channels]
    # MNIST images are 28x28 pixels, and have one color channel
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

    # Convolutional Layer #1
    # Computes 32 features using a 5x5 filter with ReLU activation.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 28, 28, 1]
    # Output Tensor Shape: [batch_size, 28, 28, 32]
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    # Pooling Layer #1
    # First max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 28, 28, 32]
    # Output Tensor Shape: [batch_size, 14, 14, 32]
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

    # Convolutional Layer #2
    # Computes 64 features using a 5x5 filter.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 14, 14, 32]
    # Output Tensor Shape: [batch_size, 14, 14, 64]
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    # Pooling Layer #2
    # Second max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 14, 14, 64]
    # Output Tensor Shape: [batch_size, 7, 7, 64]
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

    # Flatten tensor into a batch of vectors
    # Input Tensor Shape: [batch_size, 7, 7, 64]
    # Output Tensor Shape: [batch_size, 7 * 7 * 64]
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

    # Dense Layer
    # Densely connected layer with 1024 neurons
    # Input Tensor Shape: [batch_size, 7 * 7 * 64]
    # Output Tensor Shape: [batch_size, 1024]
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

    # Add dropout operation; 0.6 probability that element will be kept
    dropout = tf.layers.dropout(
        inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits layer
    # Input Tensor Shape: [batch_size, 1024]
    # Output Tensor Shape: [batch_size, 10]
    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    print(logits)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

## Code Run
MNIST data 다운로드  

### Training
Batch size: 100  
Epoch: 0  
Shuffle: True  
Step: 20000

In [5]:
def main(unused_argv):
    # Load training and eval data
    mnist = tf.contrib.learn.datasets.load_dataset("mnist")
    train_data = mnist.train.images  # Returns np.array
    train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
    eval_data = mnist.test.images  # Returns np.array
    eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

    # Create the Estimator
    mnist_classifier = tf.estimator.Estimator(
        model_fn=cnn_model_fn, model_dir="./MNIST_Test")

    # Set up logging for predictions
    # Log the values in the "Softmax" tensor with label "probabilities"
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
        tensors=tensors_to_log, every_n_iter=50)

    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": train_data},
        y=train_labels,
        batch_size=100,
        num_epochs=None,
        shuffle=True)
    mnist_classifier.train(
        input_fn=train_input_fn,
        steps=20000,
        hooks=[logging_hook])

    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": eval_data},
        y=eval_labels,
        num_epochs=1,
        shuffle=False)
    eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)

In [ ]:
if __name__ == "__main__":
    tf.app.run()

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_service': None, '_log_step_count_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff72d9e7cf8>, '_num_ps_replicas': 0, '_task_id': 0, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_task_type': 'worker', '_model_dir': './MNIST_Test', '_is_chief': True, '_tf_random_seed': None, '_keep_checkpoint_max': 5, '_session_config': None, '_num_worker_replicas': 1, '_master': '', '_keep_checkpoint_every_n_hours': 10000, '_

INFO:tensorflow:loss = 2.3174665, step = 1
INFO:tensorflow:probabilities = [[0.10218612 0.10108373 0.10807511 0.10595891 0.09264028 0.08979005
  0.10196023 0.10776134 0.08258162 0.10796261]
 [0.10679434 0.10474739 0.10109833 0.10269022 0.09737808 0.09946194
  0.10033211 0.08800715 0.09170549 0.10778493]
 [0.09385723 0.10266202 0.10142379 0.10173433 0.09358697 0.10102544
  0.11298709 0.09840757 0.09196625 0.10234929]
 [0.09612109 0.10005458 0.09860758 0.11827911 0.09952029 0.09727849
  0.09959354 0.09342211 0.09114511 0.10597803]
 [0.10017806 0.11045973 0.09307023 0.09290338 0.0997835  0.09954
  0.10314154 0.10632651 0.09767079 0.09692626]
 [0.10220003 0.11235285 0.09621383 0.09614596 0.09271957 0.09882198
  0.10552308 0.10968009 0.08805283 0.09828982]
 [0.10800835 0.11231107 0.09791018 0.09941394 0.08780782 0.11365616
  0.10506643 0.09368998 0.09351812 0.08861792]
 [0.1000426  0.09933086 0.09579244 0.1065631  0.09212714 0.10248268
  0.11488503 0.09905203 0.08660393 0.10312016]
 [0.1073

INFO:tensorflow:global_step/sec: 172.58
INFO:tensorflow:probabilities = [[0.10538572 0.10793341 0.0944215  0.0956517  0.08980274 0.10390009
  0.0942728  0.10367395 0.08998832 0.11496978]
 [0.08939116 0.10345069 0.09859865 0.10151725 0.09518413 0.10952219
  0.11286788 0.09412736 0.08450007 0.1108407 ]
 [0.09686758 0.10119551 0.09814314 0.09858802 0.10214881 0.10413388
  0.10191081 0.09890334 0.08966484 0.10844407]
 [0.10898948 0.11571074 0.10078951 0.09796703 0.09670413 0.09796762
  0.08856352 0.10480953 0.09134098 0.09715743]
 [0.09494427 0.10089743 0.09963306 0.09824495 0.1019669  0.10521918
  0.11466633 0.09659726 0.09254129 0.09528928]
 [0.11180475 0.10851195 0.09427182 0.10534804 0.09072831 0.10435838
  0.1107109  0.0903523  0.09149802 0.09241552]
 [0.09435936 0.10207592 0.09809536 0.10685624 0.09712264 0.10089995
  0.1087813  0.10333963 0.09431263 0.09415695]
 [0.09824476 0.10877252 0.09249187 0.10978587 0.09412339 0.0969874
  0.11178616 0.09337865 0.09916978 0.09525961]
 [0.10735

INFO:tensorflow:loss = 2.2954285, step = 101 (0.584 sec)
INFO:tensorflow:probabilities = [[0.10218468 0.10169958 0.11266545 0.09546817 0.10593437 0.09080572
  0.1017483  0.10098962 0.08470006 0.10380409]
 [0.10206317 0.09795033 0.10694449 0.10167632 0.09456421 0.09965184
  0.11045086 0.10037575 0.10175952 0.08456348]
 [0.11036415 0.08771794 0.09831205 0.09853223 0.10134727 0.10102153
  0.09997945 0.10652366 0.08873454 0.10746712]
 [0.09978501 0.09970007 0.08536377 0.10165336 0.10675748 0.10707073
  0.10217301 0.10100257 0.09354445 0.10294949]
 [0.09091046 0.10657658 0.10074354 0.11339983 0.09712676 0.10170373
  0.0893691  0.11230503 0.08856396 0.09930106]
 [0.09031363 0.10184861 0.09935367 0.11311525 0.09866713 0.09831479
  0.10762908 0.10303913 0.0867121  0.10100662]
 [0.10458986 0.09054624 0.11324083 0.10565571 0.09549153 0.10499997
  0.09353616 0.10051443 0.08920156 0.10222374]
 [0.09985551 0.10075644 0.09578068 0.11244226 0.10669663 0.0965858
  0.11260348 0.10691635 0.08328566 0.08

INFO:tensorflow:global_step/sec: 202.216
INFO:tensorflow:probabilities = [[0.10031202 0.09724636 0.10383042 0.09693961 0.09871473 0.09314945
  0.1083661  0.09833208 0.0951108  0.10799842]
 [0.10953096 0.10542977 0.10247812 0.0968257  0.09445921 0.10508775
  0.09331601 0.09468789 0.09087568 0.10730901]
 [0.08834592 0.11349381 0.08946358 0.09874082 0.09730756 0.09626397
  0.10961752 0.09502182 0.10023092 0.11151403]
 [0.09713559 0.11099345 0.10363732 0.10112295 0.09708855 0.09692111
  0.09421697 0.09993168 0.09499734 0.10395501]
 [0.10209548 0.10151403 0.10795575 0.10153161 0.09052499 0.09798886
  0.1096211  0.09214166 0.10357732 0.09304918]
 [0.09166806 0.10197502 0.09770712 0.09440865 0.09682725 0.09959541
  0.10408286 0.10821991 0.09364247 0.11187328]
 [0.10205282 0.10275839 0.09946345 0.10662429 0.10053874 0.10276718
  0.10499758 0.08842754 0.08262401 0.10974602]
 [0.1090493  0.11214601 0.10314209 0.10388029 0.09526999 0.09540883
  0.09344991 0.08732893 0.09301804 0.10730653]
 [0.115

...

INFO:tensorflow:loss = 0.17028163, step = 8001 (0.494 sec)
INFO:tensorflow:probabilities = [[0.00021767 0.025186   0.00301377 0.00356084 0.01776614 0.00570039
  0.7668287  0.00013511 0.17591612 0.0016751 ]
 [0.00255878 0.04788454 0.01165572 0.5088264  0.00038507 0.28806445
  0.0017282  0.00078651 0.13650839 0.00160196]
 [0.00001649 0.00000224 0.00000171 0.00000168 0.00008786 0.0000131
  0.00000007 0.9710905  0.00004359 0.02874274]
 [0.99983966 0.         0.00007786 0.00000194 0.00000003 0.00003412
  0.00004539 0.00000006 0.0000006  0.00000034]
 [0.9999931  0.         0.00000226 0.00000009 0.         0.00000153
  0.00000284 0.00000004 0.00000003 0.00000009]
 [0.00007024 0.9890376  0.00250034 0.00121351 0.00016234 0.00038193
  0.00036336 0.00352977 0.00223864 0.00050238]
 [0.02479236 0.03586713 0.22348857 0.0182901  0.00005726 0.4446843
  0.23362468 0.00035235 0.01881394 0.00002926]
 [0.00160846 0.04994778 0.00317935 0.00036599 0.03021949 0.6916532
  0.15141195 0.00038756 0.07101789 0.00

INFO:tensorflow:global_step/sec: 204.815
INFO:tensorflow:probabilities = [[0.00001008 0.00018238 0.00057434 0.00031218 0.79022515 0.0004107
  0.00009955 0.00692845 0.00090236 0.2003548 ]
 [0.00177268 0.06058452 0.00197554 0.00503125 0.08106074 0.00714491
  0.00466485 0.13841859 0.04694979 0.6523971 ]
 [0.0000006  0.00014609 0.74117804 0.0000346  0.00000286 0.0000063
  0.2586187  0.00000005 0.00001272 0.00000001]
 [0.00564957 0.00076317 0.9573904  0.0051531  0.00159154 0.00499244
  0.00114722 0.00078159 0.01939103 0.00313988]
 [0.00005587 0.9775115  0.00082955 0.00121845 0.00007512 0.0002991
  0.00005335 0.00831079 0.00497414 0.00667213]
 [0.0002602  0.00000032 0.00004634 0.00005923 0.00253102 0.000465
  0.00000193 0.9617139  0.00002469 0.03489738]
 [0.9994943  0.00000001 0.00002015 0.00003676 0.00000036 0.00037673
  0.00002793 0.00003332 0.00000481 0.00000561]
 [0.9974299  0.00000026 0.00003118 0.00005552 0.00000018 0.00206256
  0.00010743 0.00002868 0.00007058 0.00021372]
 [0.00002236

INFO:tensorflow:loss = 0.25365925, step = 8101 (0.488 sec)
INFO:tensorflow:probabilities = [[0.00000621 0.00006809 0.00008991 0.99641824 0.00000318 0.00250712
  0.00000086 0.00005894 0.0002571  0.00059025]
 [0.00293778 0.04118858 0.01378867 0.00468418 0.00169748 0.08854637
  0.00495805 0.00205028 0.8389006  0.00124797]
 [0.0000076  0.00000439 0.9998592  0.00007867 0.0000005  0.00000551
  0.00000369 0.00000548 0.00003327 0.00000171]
 [0.00000251 0.00044361 0.00001663 0.00093044 0.00542976 0.00010647
  0.00000083 0.03737633 0.00644361 0.9492498 ]
 [0.00007275 0.988247   0.00585906 0.00028514 0.00099584 0.00013711
  0.0000594  0.00155088 0.00272585 0.00006693]
 [0.00004693 0.00085829 0.0000783  0.00071266 0.00358009 0.9901121
  0.0005691  0.0001227  0.00374588 0.00017387]
 [0.00000006 0.         0.00000001 0.00005253 0.00000006 0.0000059
  0.         0.9997044  0.00000033 0.00023673]
 [0.00001669 0.00000068 0.00000939 0.0001883  0.05717111 0.00005102
  0.00000305 0.00497511 0.00019127 0.9

INFO:tensorflow:global_step/sec: 201.52
INFO:tensorflow:probabilities = [[0.00584876 0.00100749 0.00175256 0.0744152  0.08396033 0.6518795
  0.05526369 0.0005236  0.02501577 0.1003332 ]
 [0.00033369 0.9742503  0.00521434 0.00193855 0.00429794 0.00076948
  0.00164267 0.00642    0.00227305 0.00285992]
 [0.00000079 0.00000093 0.00000299 0.00003853 0.00000143 0.00000468
  0.         0.99574786 0.00000667 0.00419623]
 [0.09854328 0.00009984 0.00019945 0.8297483  0.00001479 0.06095369
  0.00007105 0.00118247 0.00226628 0.00692099]
 [0.00124174 0.0001983  0.00487347 0.00062435 0.35236043 0.00192728
  0.00959135 0.0394101  0.00394039 0.5858326 ]
 [0.00000544 0.00001205 0.9997402  0.00011883 0.00000049 0.00002062
  0.00000507 0.00002769 0.00006459 0.00000504]
 [0.0001692  0.00001223 0.00001548 0.00034525 0.7212306  0.00023934
  0.00043283 0.00102061 0.00007203 0.27646238]
 [0.00054683 0.00001835 0.00005225 0.00051428 0.00012139 0.00028961
  0.00000315 0.971566   0.00015039 0.02673769]
 [0.04010

INFO:tensorflow:loss = 0.38579908, step = 8201 (0.497 sec)
INFO:tensorflow:probabilities = [[0.00149337 0.45500663 0.02111563 0.39256677 0.00075227 0.03856043
  0.00094664 0.02318376 0.06125153 0.00512298]
 [0.00000164 0.         0.00020776 0.99836916 0.         0.00106435
  0.         0.00000014 0.00035635 0.00000055]
 [0.23920342 0.00000565 0.7200834  0.00015664 0.00000104 0.00517492
  0.00074514 0.00001681 0.03458598 0.00002696]
 [0.9998148  0.         0.00000308 0.00000668 0.         0.0001681
  0.00000595 0.00000047 0.0000003  0.00000064]
 [0.00000006 0.         0.00000001 0.00000089 0.         0.00000012
  0.         0.99997747 0.00000002 0.00002138]
 [0.00307397 0.0004392  0.00067072 0.00016223 0.81141025 0.00030544
  0.02091321 0.00163725 0.00126617 0.1601216 ]
 [0.00001057 0.0000567  0.993856   0.00527917 0.00007458 0.00004653
  0.0000019  0.00000225 0.00056986 0.00010237]
 [0.00000114 0.00007369 0.00029589 0.00071956 0.00001247 0.00000143
  0.00000004 0.98996407 0.00001675 0.